In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import word_tokenize, FreqDist, BigramCollocationFinder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string


### Objective:
Guide is: [article](https://towardsdatascience.com/sentiment-analysis-and-product-recommendation-on-amazons-electronics-dataset-reviews-part-1-6b340de660c2)
Set up a 'dev doc' 

The code below comes from where I got the data. It was a helper function provided to help with ease of access.

In [2]:
import pandas as pd
import gzip
import json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')
def clean_song(song):
    cleaned_song = []
    for line in song:
        if not '[' in line and not ']' in line:
            for symbol in ",.'?!''\n":
                line = line.replace(symbol, "").lower()
            cleaned_song.append(line)
    return cleaned_song
def getContractions(series):
    contr = set()
    for row in series:
        for word in row.split(' '):
            if "'" in word:
                contr.add(word)
    return contr

In [3]:
df = getDF('Magazine_Subscriptions.json.gz')

In [4]:
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,5.0,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN
1,5.0,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN
2,3.0,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN
3,5.0,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN
4,5.0,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89689 entries, 0 to 89688
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         89689 non-null  float64
 1   vote            24103 non-null  object 
 2   verified        89689 non-null  bool   
 3   reviewTime      89689 non-null  object 
 4   reviewerID      89689 non-null  object 
 5   asin            89689 non-null  object 
 6   reviewerName    89687 non-null  object 
 7   reviewText      89656 non-null  object 
 8   summary         89670 non-null  object 
 9   unixReviewTime  89689 non-null  int64  
 10  style           51398 non-null  object 
 11  image           135 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 8.3+ MB


In [7]:
to_drop = ['image', 'style']
df.vote.fillna(0, inplace=True)
df.reviewerName.fillna('Amazon Customer', inplace=True)
df.drop(to_drop, axis=1, inplace=True)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89640 entries, 0 to 89688
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         89640 non-null  float64
 1   vote            89640 non-null  object 
 2   verified        89640 non-null  bool   
 3   reviewTime      89640 non-null  object 
 4   reviewerID      89640 non-null  object 
 5   asin            89640 non-null  object 
 6   reviewerName    89640 non-null  object 
 7   reviewText      89640 non-null  object 
 8   summary         89640 non-null  object 
 9   unixReviewTime  89640 non-null  int64  
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 6.9+ MB


In [8]:
clean_text = clean_song(df.reviewText)

In [9]:
results = {}
for column in df.columns:
    results[column] = df[column].value_counts()
results

{'overall': 5.0    53754
 4.0    12667
 1.0    11027
 3.0     6969
 2.0     5223
 Name: overall, dtype: int64,
 'vote': 0      65541
 2       6317
 3       3657
 4       2403
 5       1752
        ...  
 124        1
 228        1
 248        1
 252        1
 271        1
 Name: vote, Length: 288, dtype: int64,
 'verified': True     58625
 False    31015
 Name: verified, dtype: int64,
 'reviewTime': 02 20, 2015    146
 03 29, 2016    131
 03 25, 2014    127
 03 1, 2016     113
 05 30, 2015    113
               ... 
 07 11, 2018      1
 08 26, 2005      1
 03 26, 2007      1
 02 28, 2002      1
 12 1, 2006       1
 Name: reviewTime, Length: 5804, dtype: int64,
 'reviewerID': A3JPFWKS83R49V    55
 A2OTUWUSH49XIN    26
 AEMZRE6QYVQBS     25
 A3GA09FYFKL4EY    24
 A3R7MXVQRGGIQ9    22
                   ..
 AN6FBEDZQSAQN      1
 A37I16H7BDMW18     1
 AXSQHMIGA3RF6      1
 A3PDVMWMRUJW31     1
 ATROQ9AVYMCDE      1
 Name: reviewerID, Length: 72063, dtype: int64,
 'asin': B00005NIOH    1718

In [10]:
stopwords_list = stopwords.words('english') + list(string.punctuation) + ["''", '""', '...', '``', "'s", "n't"]
def process_article(article):
    return [w.lower() for w in word_tokenize(article) if w.lower() not in stopwords_list]

In [11]:
data = clean_text

In [12]:
processed_data = list(map(process_article, data))


In [13]:
out = False
if 'not' in stopwords_list:
    out = True
out

True

In [21]:
processed_data

[['computer',
  'enthusiast',
  'maxpc',
  'welcome',
  'sight',
  'mailbox',
  'remember',
  'years',
  'savorying',
  'every',
  'page',
  'boot',
  'called',
  'beginning',
  'still',
  'obcessed',
  'pcs',
  'anyone',
  'advanced',
  'users',
  'beginners',
  'looking',
  'knowledge',
  'profit',
  'every',
  'issue',
  'maxpc',
  'icing',
  'cake',
  'subscription',
  'comes',
  'cd-rom',
  'packed',
  'demos',
  'utilities',
  'useful',
  'apps',
  'helpful',
  'blessed',
  'broadband',
  'connections',
  'discovered',
  'community',
  'hardware',
  'enthusiast',
  'web',
  'sites',
  'maxpc',
  'formerly',
  'boot',
  'really',
  'informative',
  'source',
  'computing',
  'news',
  'articles',
  'day',
  'consider',
  'subscription',
  'worth',
  '10',
  'subscriptions',
  'computing',
  'mags',
  'cant',
  'wait',
  'merge',
  'dvd',
  'media',
  'maybe',
  'end',
  'offering',
  'info',
  'divx',
  'codecs',
  'encoding',
  'movies',
  'best',
  'bang',
  'buck',
  'audio',
 

In [14]:
total_vocab = set()
for article in processed_data:
    total_vocab.update(article)
len(total_vocab)

78065

In [15]:
review_concat = []
for review in processed_data:
    review_concat += review
review_freqdist = FreqDist(review_concat)
review_freqdist.most_common(200)
total_word_count = sum(review_freqdist.values())

In [16]:
normalized_freqs = {}
for word in review_freqdist.most_common(200):
    normalized_freq = word[1]/total_word_count
    normalized_freqs[word[0]] = normalized_freq
normalized_freqs

{'magazine': 0.03354613410247565,
 'great': 0.011498637132898412,
 'articles': 0.008845339278179386,
 'love': 0.008397044394363742,
 'like': 0.008227350850772511,
 'subscription': 0.008046513253572302,
 'read': 0.00797002452198342,
 'good': 0.007685851154954519,
 'issue': 0.007129661570288332,
 'one': 0.006852073325846556,
 'get': 0.005704235804121983,
 'would': 0.005098404526106719,
 'magazines': 0.004767122205185328,
 'years': 0.00473875552327157,
 'time': 0.0043558053174358345,
 'reading': 0.004303124336738855,
 'new': 0.004166356406083235,
 'kindle': 0.004142548655191331,
 'always': 0.003929798540838144,
 'many': 0.003908523529402826,
 'really': 0.003896872927902532,
 'dont': 0.0037205942617241777,
 'every': 0.0037018519897454444,
 'much': 0.003617258491895487,
 'recipes': 0.0035888918099817287,
 'year': 0.0034961935458706975,
 'well': 0.0034784643696745988,
 'issues': 0.003438447086260547,
 'first': 0.0034212244579557654,
 'price': 0.0033244738107141973,
 'also': 0.003253557105929

In [17]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(review_concat)
scored = finder.score_ngrams(bigram_measures.raw_freq)
scored[:50]

[(('great', 'magazine'), 0.002344810188906905),
 (('love', 'magazine'), 0.002257177403709045),
 (('first', 'issue'), 0.0011893744488125758),
 (('magazine', 'great'), 0.001148850617507207),
 (('look', 'forward'), 0.0008732885646306987),
 (('good', 'magazine'), 0.0008677165378262105),
 (('every', 'month'), 0.0007608349327583001),
 (('every', 'issue'), 0.0007441188523448355),
 (('like', 'magazine'), 0.0006650973812993662),
 (('magazine', 'love'), 0.0005946872244062879),
 (('cover', 'cover'), 0.0005936741286236537),
 (('years', 'ago'), 0.0005789842397754575),
 (('kindle', 'fire'), 0.000558722324122773),
 (('great', 'articles'), 0.0005425127916006255),
 (('year', 'old'), 0.0005404866000353571),
 (('well', 'written'), 0.0005252901632958437),
 (('magazine', 'good'), 0.0005116133702302817),
 (('great', 'price'), 0.0005070544392084277),
 (('interesting', 'articles'), 0.0004928710982515487),
 (('many', 'years'), 0.0004797008530773038),
 (('highly', 'recommend'), 0.0004791943051859867),
 (('read'

In [18]:
pmi_finder = BigramCollocationFinder.from_words(review_concat)
pmi_finder.apply_freq_filter(5)
review_pmi_scored = pmi_finder.score_ngrams(bigram_measures.pmi)
review_pmi_scored[:50]

[(('a-section', 'a-spacing-small'), 18.327835489134387),
 (('a-spacing-small', 'a-spacing-top-mini'), 18.327835489134387),
 (('a-spacing-top-mini', 'video-block'), 18.327835489134387),
 (('dame', 'edna'), 18.327835489134387),
 (('settimana', 'enigmistica'), 18.327835489134387),
 (('video-block', '/div'), 18.327835489134387),
 (('humpty', 'dumpty'), 18.105443067797943),
 (('bells', 'whistles'), 18.10544306779794),
 (('div', 'id='), 18.10544306779794),
 (('christina', 'aguilera'), 18.105443067797935),
 (('miley', 'cyrus'), 18.105443067797935),
 (('bona', 'fide'), 18.06480108330059),
 (('womennice', 'womennice'), 18.06480108330059),
 (('roz', 'chast'), 17.912797989855545),
 (('stanley', 'schmidt'), 17.842408661964143),
 (('movers', 'shakers'), 17.742872988413232),
 (('katy', 'perry'), 17.74287298841323),
 (('suze', 'orman'), 17.74287298841323),
 (('puerto', 'rico'), 17.690405568519093),
 (('fareed', 'zakaria'), 17.59086989496818),
 (('tai', 'chi'), 17.59086989496818),
 (('rue', 'morgue'),